In [34]:
%%writefile scaling_models.py
import abc
class Scaler(abc.ABC):
    @abc.abstractmethod
    def fit(self, s1: float, q1: float, s2: float, q2: float):
        pass
    @abc.abstractmethod
    def predict(self, s_new: float) -> float:
        pass
class RatioBasedScaler(Scaler):
    def __init__(self):
        self.k_avg = 0.0

    def fit(self, s1: float, q1: float, s2: float, q2: float):
        if s1 == 0 or s2 == 0:
            raise ValueError("Serving size cannot be zero.")
        k1 = q1 / s1
        k2 = q2 / s2
        self.k_avg = (k1 + k2) / 2.0

    def predict(self, s_new: float) -> float:
        return self.k_avg * s_new
class LinearScaler(Scaler):
    def __init__(self):
        self.m = 0.0  # slope
        self.c = 0.0  # y-intercept

    def fit(self, s1: float, q1: float, s2: float, q2: float):
        if s1 == s2:
            raise ValueError("Serving sizes for fitting must be different.")
        self.m = (q2 - q1) / (s2 - s1)
        self.c = q1 - self.m * s1

    def predict(self, s_new: float) -> float:
        return self.m * s_new + self.c

Overwriting scaling_models.py


In [35]:
%%writefile evaluation.py
from typing import List

def mean_absolute_error(actuals: List[float], predictions: List[float]) -> float:
    if len(actuals) != len(predictions):
        raise ValueError("Lists must have the same length.")
    return sum(abs(a - p) for a, p in zip(actuals, predictions)) / len(actuals)

def mean_absolute_percentage_error(actuals: List[float], predictions: List[float]) -> float:
    if len(actuals) != len(predictions):
        raise ValueError("Lists must have the same length.")
    total_percentage_error = 0.0
    count = 0
    for a, p in zip(actuals, predictions):
        if a > 0:
            total_percentage_error += abs((a - p) / a)
            count += 1
    return (total_percentage_error / count) * 100.0 if count > 0 else 0.0

Overwriting evaluation.py


In [36]:
%%writefile main.py
import json
import re
import itertools
from collections import defaultdict
from typing import Dict, List, Tuple

from scaling_models import Scaler, RatioBasedScaler, LinearScaler
from evaluation import mean_absolute_error, mean_absolute_percentage_error

class RecipeBook:
    def __init__(self, filepath: str):
        self.recipes = self._load_recipes_from_file(filepath)

    @staticmethod
    def _parse_grams(quantity_text: str) -> float | None:
        match = re.search(r'(\d+(?:\.\d+)?)\s*grams', quantity_text, re.IGNORECASE)
        return float(match.group(1)) if match else None

    def _load_recipes_from_file(self, filepath: str) -> Dict:
        try:
            with open(filepath, 'r') as f:
                raw_data = json.load(f)
        except FileNotFoundError:
            print(f"Error: The recipe file at {filepath} was not found.")
            return {}

        parsed_recipes = {}
        for dish, servings_data in raw_data.items():
            parsed_recipes[dish] = defaultdict(dict)
            for s_size, ingredients in servings_data.items():
                for name, qty_text in ingredients.items():
                    grams = self._parse_grams(qty_text)
                    if grams is not None:
                        parsed_recipes[dish][name][int(s_size)] = grams
        return parsed_recipes

    def list_dishes(self) -> List[str]:
        return list(self.recipes.keys())

    def list_ingredients_for_dish(self, dish: str) -> List[str]:
        return list(self.recipes.get(dish, {}).keys())


class RecipeScaler:
    def __init__(self, book: RecipeBook):
        self.book = book
        self.models: Dict[str, Scaler] = {
            "Ratio-Based Scaling": RatioBasedScaler(),
            "Linear Interpolation": LinearScaler()
        }

    def evaluate_models(self, serving_sizes: List[int] = [1, 2, 3, 4]):
        print("--- Model Performance Evaluation ---")
        results = defaultdict(lambda: {'mae': [], 'mape': []})
        training_pairs = list(itertools.combinations(serving_sizes, 2))

        for dish, ingredients in self.book.recipes.items():
            for name, quantities in ingredients.items():
                if len(quantities) != len(serving_sizes):
                    continue
                for s1, s2 in training_pairs:
                    test_sizes = sorted(list(set(serving_sizes) - {s1, s2}))
                    q1, q2 = quantities[s1], quantities[s2]
                    actuals = [quantities[s] for s in test_sizes]
                    for model_name, model in self.models.items():
                        model.fit(s1, q1, s2, q2)
                        predictions = [model.predict(s) for s in test_sizes]
                        results[model_name]['mae'].append(mean_absolute_error(actuals, predictions))
                        results[model_name]['mape'].append(mean_absolute_percentage_error(actuals, predictions))
        
        self._print_evaluation_results(results)

    def _print_evaluation_results(self, results: dict):
        print(f"\n{'Model':<25} | {'Average MAE (grams)':<20} | {'Average MAPE (%)':<20}")
        print("-" * 70)
        for name, res in results.items():
            avg_mae = sum(res['mae']) / len(res['mae'])
            avg_mape = sum(res['mape']) / len(res['mape'])
            print(f"{name:<25} | {avg_mae:<20.4f} | {avg_mape:<20.4f}")

    def get_scaled_ingredient(self, dish: str, ingredient: str, new_servings: List[float]):
        quantities = self.book.recipes.get(dish, {}).get(ingredient)

        if not quantities or len(quantities) < 2:
            print(f"Sorry, not enough data to scale '{ingredient}' in '{dish}'.")
            return

        min_s = min(quantities.keys())
        max_s = max(quantities.keys())
        
        scaler = LinearScaler()
        scaler.fit(min_s, quantities[min_s], max_s, quantities[max_s])
        
        print(f"To scale '{ingredient}', I'm using the quantities for {min_s} and {max_s} servings as a reference.")
        print("\nHere are your results:")
        for s_new in new_servings:
            prediction = scaler.predict(s_new)
            print(f"  - For {s_new} servings, you will need {prediction:.2f} grams.")


def run_interactive_tool():
    print("Welcome to the Interactive Recipe Scaler!")
    
    file_path = '/kaggle/input/paneer/paneer_recipes.json'
    recipe_book = RecipeBook(file_path)
    scaler = RecipeScaler(recipe_book)
    
    scaler.evaluate_models()
    
    print(f"\nAvailable dishes: {recipe_book.list_dishes()}")
    dish = input("Which dish would you like to make? ").strip()

    ingredients = recipe_book.list_ingredients_for_dish(dish)
    if not ingredients:
        print(f"Sorry, I couldn't find that dish. Please try again.")
        return
        
    print(f"Available ingredients: {ingredients}")
    ingredient = input("Which ingredient do you want to scale? ").strip()

    while True:
        try:
            servings_str = input("How many servings do you need? (you can enter multiple, separated by commas): ")
            target_servings = [float(s.strip()) for s in servings_str.split(',')]
            break
        except ValueError:
            print("That doesn't look right. Please enter numbers, like '7.5' or '8, 12'.")

    scaler.get_scaled_ingredient(dish, ingredient, target_servings)
    print("\nHappy cooking!")

if __name__ == '__main__':
    run_interactive_tool()

Overwriting main.py


In [37]:
%run main.py

Welcome to the Interactive Recipe Scaler!
--- Model Performance Evaluation ---

Model                     | Average MAE (grams)  | Average MAPE (%)    
----------------------------------------------------------------------
Ratio-Based Scaling       | 13.4196              | 24.9697             
Linear Interpolation      | 6.2657               | 15.9643             

Available dishes: ['palak_paneer', 'shahi_paneer', 'matar_paneer', 'paneer_masala']


Which dish would you like to make?  palak_paneer


Available ingredients: ['Onion', 'Garlic', 'Green chilli', 'BB Royal Bay leaf', 'BB Royal Cinnamon', 'BB Royal Green cardamom', "Mother's Recipe Ginger garlic paste", 'Tomato', 'Blanched spinach puree', 'Paneer']


Which ingredient do you want to scale?  Garlic
How many servings do you need? (you can enter multiple, separated by commas):  2


To scale 'Garlic', I'm using the quantities for 1 and 4 servings as a reference.

Here are your results:
  - For 2.0 servings, you will need 15.00 grams.

Happy cooking!
